In [4]:
!pip install mlflow datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00


In [25]:
# Importación de librerias

import mlflow
from mlflow.models import infer_signature

import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder

## Información del dataset

Here’s the table in Markdown format:  


| Feature                         | Type   |
|----------------------------------|--------|
| is_male                         | bool   |
| ethnicity                       | string |
| parental_level_of_education     | int8   |
| has_standard_lunch              | bool   |
| has_completed_preparation_test  | bool   |
| reading_score                   | int64  |
| writing_score                   | int64  |
| math_score                      | int64  |


In [21]:
# Carga del dataset de resultados de matemáticas

dataset = load_dataset("mstz/student_performance", "math")
dataset = dataset['train'].to_pandas()

#Validación de la carga del dataset

dataset.head()


,is_male,ethnicity,parental_level_of_education,has_standard_lunch,has_completed_preparation_test,reading_score,writing_score,has_passed_math_exam
0,False,group D,2,True,True,70,78,0
1,True,group D,5,True,False,93,87,1
2,False,group D,2,False,False,76,77,0
3,True,group B,2,False,False,70,63,1
4,False,group D,5,True,False,85,86,1


In [ ]:
# Definición de las variables de entrada y salida
X=dataset.drop('has_passed_math_exam',axis=1)
y=dataset['has_passed_math_exam']


In [29]:
# Codificación de las variables categoricas

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

categoricas = ['ethnicity', 'parental_level_of_education']

Xenco = encoder.fit_transform(X[categoricas])
Xenco_df = pd.DataFrame(Xenco , columns=encoder.get_feature_names_out(categoricas))

# Reemplazo de las variables originales por las codificadas
X = X.drop(categoricas, axis=1)
X = pd.concat([X, Xenco_df], axis=1)

In [30]:
# Separación del dataset para entrenamiento y validación

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Definición del modelo de DecisionTree para la predicción de la variable y

params = {
    'max_depth': 5,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}

modelo = DecisionTreeClassifier(**params) #Iniciación del modelo
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test) #Predicción de los datos de prueba

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


# Monitoreo con Ml-Flow